In [ ]:
import torch
from lib.optimizer.pcg import conjugate_gradient

N = 400
_A = torch.rand((N, N))
A = torch.sqrt(_A.T @ _A) # positive semidefinite and symmetric
b = torch.rand((N))
# x_pcg  = conjugate_gradient(A, b, verbose=True, max_iter=100)
# x_optim= torch.linalg.solve(A, b) 
# pcg_norm = ((A @ x_pcg) - b).norm()
# optim_norm  = ((A @ x_optim) - b).norm()
# print(pcg_norm, optim_norm)

In [ ]:
import timeit
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

t = 100
A = A.to("cuda")
b = b.to("cuda")


def foo():
    conjugate_gradient(A, b, verbose=True, max_iter=5)
    # torch.linalg.solve(A, b)


with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=False
) as prof:
    # foo()
    timeit.timeit(foo, number=t) / t

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

In [ ]:
from lib.optimizer.pcg import conjugate_gradient

def conjugate_gradient(
    A: torch.Tensor,  # dim (N,N)
    b: torch.Tensor,  # dim (N)
    x0: torch.Tensor | None = None,  # dim (N)
    max_iter: int = 20,
    verbose: bool = False,
    tol: float = 1e-08,
):
    k = 0
    converged = False

    xk = torch.zeros_like(b) if x0 is None else x0  # (N)
    rk = b - A @ xk  # column vector (N)
    pk = rk

    if torch.norm(rk) < tol:
        converged = True

    while k < max_iter and not converged:
        
        # compute step size
        ak = (rk[None] @ rk) / (pk[None] @ A @ pk)
        # update unknowns
        xk_1 = xk + ak * pk
        # compute residuals
        rk_1 = rk - ak * A @ pk
        # compute new pk
        bk = (rk_1[None] @ rk_1) / (rk[None] @ rk)
        pk_1 = rk_1 + bk * pk
        # update the next stateprint
        xk = xk_1
        pk = pk_1
        rk = rk_1

        k += 1
        if torch.norm(rk) < tol:
            converged = True

    return xk

t = 1000
N = 6
_A = torch.rand((N, N))
A = torch.sqrt(_A.T @ _A).to("cuda") # positive semidefinite and symmetric
b = torch.rand((N)).to("cuda")

def foo():
    conjugate_gradient(A, b, max_iter=4)
    # torch.linalg.solve(A,b) 

print((timeit.timeit(foo, number=t) / t ) * 1000)

In [ ]:
torch.linalg.cond(A)

In [ ]:
import torch
out = torch.load("/home/borth/GuidedResearch/logs/2024-07-04_09-39-30_optimize/linsys/0000000.pt")
A = out["A"]
b = out["b"]
x = out["x"]

In [ ]:
torch.linalg.cond(A)

In [ ]:
from lib.optimizer.pcg import JaccobiConditionNet
j = JaccobiConditionNet()
M = j(A)
torch.linalg.cond(M @ A)

In [ ]:
from tqdm import tqdm
from lib.optimizer.pcg import PCGSolver
from lib.optimizer.pcg import conjugate_gradient
from torch.optim import Adam, SGD
import torch

# define the matrixes

def generate_data(N, x_eps= 1e-02, a_eps=1e-01):
    E = torch.rand((N, N)) * a_eps
    A = torch.eye(N) + (E.T @ E) 
    x_gt = torch.rand((N))  * x_eps
    b = A @ x_gt
    return A, b, x_gt

def eval_loss(x_pcg, x_gt, verbose=True):
    l_pcg = (x_pcg-x_gt).norm()
    if verbose:
        print("Loss:", l_pcg.item())
    return l_pcg

torch.manual_seed(42)
N = 6
# A, b, x_gt = generate_data(N)
max_steps = 1000
lr = 1e-06
max_iter=1
verbose=True
tol=1e-08

out = torch.load("/home/borth/GuidedResearch/logs/2024-07-04_09-39-30_optimize/linsys/0000000.pt")
A = out["A"]
b = out["b"]
x_gt = torch.linalg.solve(A, b)

pcg = PCGSolver(dim=N, max_iter=max_iter, verbose=verbose, tol=tol, mode="dense")
optimizer = SGD(pcg.parameters(), lr=lr, momentum=0.90)
# optimizer = Adam([M], lr=lr)

for step in (pbar:=tqdm(range(max_steps), total=max_steps)):
    pbar.set_description(f"{step}/{max_steps}")

    optimizer.zero_grad()
  
    A, b, x_gt = generate_data(N)
    M = pcg.condition_net(A)
    C_m = torch.linalg.cond(M @ A).item()
    C_a = torch.linalg.cond(A).item()
    x = pcg(A, b)
    loss = eval_loss(x, x_gt, verbose=False) 
    loss.backward()
    optimizer.step()

    pbar.set_postfix({"loss": loss.item(), "C_m": C_m, "C_a": C_a})
    pbar.update(1)

In [ ]:
pcg = PCGSolver(dim=N, max_iter=max_iter, verbose=verbose, tol=tol, mode="dense")
A, b, x_gt = generate_data(N)
M = pcg.condition_net(A)
M

In [ ]:
torch.linalg.cond(pcg.condition_net(A) @ A)

In [ ]:
from lib.optimizer.pcg import preconditioned_conjugate_gradient
A, b, x_gt = generate_data(N)
x_cg = preconditioned_conjugate_gradient(A, b, max_iter=2)
pcg.max_iter=2
x = pcg(A, b)
loss = eval_loss(x, x_gt, verbose=True) 
loss = eval_loss(x_cg, x_gt, verbose=True) 

In [ ]:
len(list(p.size() for p in pcg.condition_net.parameters()))

In [ ]:
from lib.optimizer.pcg import conjugate_gradient
torch.manual_seed(42)
N = 100
E = torch.rand((N, N)) * 1e-02
A= torch.eye(N) + (E.T @ E)
x = torch.rand((N))  * 1e-02
b = A @ x

x_pcg = conjugate_gradient(A, b, max_iter=1)
r_pcg = (A @ x_pcg - b).mean()
r_pcg

In [ ]:
N = 5
tri_N = ((N * N - N) // 2) + N
L = torch.zeros((N, N))
tril_indices = torch.tril_indices(row=N, col=N, offset=0)
L[tril_indices[0], tril_indices[1]] = torch.rand(tri_N)
L[4, 0] = 0

In [ ]:
L.inverse()

In [ ]:
import torch
import time


def cg_batch(A_bmm, B, M_bmm=None, X0=None, rtol=1e-3, atol=0., maxiter=None, verbose=False):
    """Solves a batch of PD matrix linear systems using the preconditioned CG algorithm.

    This function solves a batch of matrix linear systems of the form

        A_i X_i = B_i,  i=1,...,K,

    where A_i is a n x n positive definite matrix and B_i is a n x m matrix,
    and X_i is the n x m matrix representing the solution for the ith system.

    Args:
        A_bmm: A callable that performs a batch matrix multiply of A and a K x n x m matrix.
        B: A K x n x m matrix representing the right hand sides.
        M_bmm: (optional) A callable that performs a batch matrix multiply of the preconditioning
            matrices M and a K x n x m matrix. (default=identity matrix)
        X0: (optional) Initial guess for X, defaults to M_bmm(B). (default=None)
        rtol: (optional) Relative tolerance for norm of residual. (default=1e-3)
        atol: (optional) Absolute tolerance for norm of residual. (default=0)
        maxiter: (optional) Maximum number of iterations to perform. (default=5*n)
        verbose: (optional) Whether or not to print status messages. (default=False)
    """
    K, n, m = B.shape

    if M_bmm is None:
        M_bmm = lambda x: x
    if X0 is None:
        X0 = M_bmm(B)
    if maxiter is None:
        maxiter = 5 * n

    assert B.shape == (K, n, m)
    assert X0.shape == (K, n, m)
    assert rtol > 0 or atol > 0
    assert isinstance(maxiter, int)

    X_k = X0
    R_k = B - A_bmm(X_k)
    Z_k = M_bmm(R_k)

    P_k = torch.zeros_like(Z_k)

    P_k1 = P_k
    R_k1 = R_k
    R_k2 = R_k
    X_k1 = X0
    Z_k1 = Z_k
    Z_k2 = Z_k

    B_norm = torch.norm(B, dim=1)
    stopping_matrix = torch.max(rtol*B_norm, atol*torch.ones_like(B_norm))

    if verbose:
        print("%03s | %010s %06s" % ("it", "dist", "it/s"))

    optimal = False
    start = time.perf_counter()
    for k in range(1, maxiter + 1):
        start_iter = time.perf_counter()
        Z_k = M_bmm(R_k)

        if k == 1:
            P_k = Z_k
            R_k1 = R_k
            X_k1 = X_k
            Z_k1 = Z_k
        else:
            R_k2 = R_k1
            Z_k2 = Z_k1
            P_k1 = P_k
            R_k1 = R_k
            Z_k1 = Z_k
            X_k1 = X_k
            denominator = (R_k2 * Z_k2).sum(1)
            denominator[denominator == 0] = 1e-8
            beta = (R_k1 * Z_k1).sum(1) / denominator
            P_k = Z_k1 + beta.unsqueeze(1) * P_k1

        denominator = (P_k * A_bmm(P_k)).sum(1)
        denominator[denominator == 0] = 1e-8
        alpha = (R_k1 * Z_k1).sum(1) / denominator
        X_k = X_k1 + alpha.unsqueeze(1) * P_k
        R_k = R_k1 - alpha.unsqueeze(1) * A_bmm(P_k)
        end_iter = time.perf_counter()

        residual_norm = torch.norm(A_bmm(X_k) - B, dim=1)

        if verbose:
            print("%03d | %8.4e %4.2f" %
                  (k, torch.max(residual_norm-stopping_matrix),
                    1. / (end_iter - start_iter)))

        if (residual_norm <= stopping_matrix).all():
            optimal = True
            break

    end = time.perf_counter()

    if verbose:
        if optimal:
            print("Terminated in %d steps (reached maxiter). Took %.3f ms." %
                  (k, (end - start) * 1000))
        else:
            print("Terminated in %d steps (optimal). Took %.3f ms." %
                  (k, (end - start) * 1000))


    info = {
        "niter": k,
        "optimal": optimal
    }

    return X_k, info


A = torch.tensor([[4.0,1],[1, 3]])[None, ...]
def A_bmm(X):
    Y = [(A[i]@X[i]).unsqueeze(0) for i in range(1)]
    return torch.cat(Y, dim=0)
b = torch.tensor([1.0,2])[None, ...][..., None]
cg_batch(A_bmm=A_bmm, B=b)

In [ ]:
N = 100
_A = torch.rand((N, N))
z = torch.zeros_like(_A)
for idx in range(70):
    i = idx * 10
    j = (idx+1)*10
    z[i:j, i:j] = 1.0
_A *= z
A = torch.sqrt(_A.T @ _A) # positive and symetric
# A = torch.diag(torch.diag(A))
b = torch.rand((N)) + 2
def A_bmm(X):
    Y = [(A[None, ...][i]@X[i]).unsqueeze(0) for i in range(1)]
    return torch.cat(Y, dim=0)


In [ ]:
import time
s = time.time()
out = cg_batch(A_bmm=A_bmm, B=b[None, ...][..., None], maxiter=10)
print(time.time() - s)
x_pcg = out[0].squeeze()
s = time.time()
x_optim = torch.linalg.solve(A, b)
print(time.time() - s)
pcg_norm = ((A @ x_pcg) - b).norm()
optim_norm = ((A @ x_optim) - b).norm()
pcg_norm, optim_norm, b.norm()

In [ ]:
torch.tensor()